In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
import numpy as np
from graphdatascience import GraphDataScience
from py2neo import Graph
from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
import neo4jupyter
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops, to_dense_adj, dense_to_sparse
from torch_geometric.nn import VGAE,GAE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from torch_geometric.utils import to_dense_adj
#neo4jupyter.init_notebook_mode()
import tqdm as notebook_tqdm

In [3]:
import pickle
# Load the list of arrays from the .pkl file
with open('med_node_embeddings_50d.pkl', 'rb') as pkl_file:
    extracted_data = pickle.load(pkl_file)

In [4]:
node_embeddings = extracted_data

In [ ]:
diseases=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR_med\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Disease.csv')
medication=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR_med\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Medication.csv')

In [ ]:
diseases_nodes=list(diseases.fname)
medication_nodes=list(medication.fname)


In [ ]:
diseases_embeddings = {key:node_embeddings[key] for key in diseases_nodes if key in node_embeddings}
medication_embeddings = {key:node_embeddings[key] for key in medication_nodes if key in node_embeddings}

In [236]:
HF_ind = ["Adomiparin", "Heparin, bovine", "Parnaparin", "Evolocumab", "Bivalirudin", "Prasugrel", "Esmolol", "Fondaparinux", "Reteplase"
          ,"Nitroglycerin", "Phenprocoumon", "Alteplase", "Ardeparin", "Irbesartan", "Eptifibatide", "Tirofiban", "Dalteparin", "Dicoumarol", "Abciximab"
          ,"Tenecteplase", "Heparin", "Alirocumab", "Atenolol", "Morphine", "Ticagrelor", "Vorapaxar", "Candesartan", "Captopril", "Lisinopril", 
          "Quinapril", "Acetylsalicylic acid", "Enalapril", "Metoprolol", "Clopidogrel", "Valsartan", "Losartan"]


In [237]:
HF_cont = ["Metamfetamine", "Testosterone cypionate", "Testosterone enanthate","Sildenafil", "Saxagliptin", "Levodopa", "Furosemide", "Phenobarbital", "Guanfacine", "Flecainide", "Indomethacin", "Lovastatin", "Progesterone", 
 "Testosterone", "Simvastatin", "Dipyridamole", "Pioglitazone", "Nefazodone", "Paclitaxel", "Phenytoin", "Liothyronine", "Levothyroxine", "Diclofenac", 
 "Estradiol", "Meclofenamic acid", "Estradiol cypionate", "Estradiol valerate", "Digoxin", "Ribavirin", "Inositol nicotinate", "Levomefolic acid", 
 "Acyclovir", "Sorafenib", "Metformin", "Sodium citrate", "Inositol"]


In [238]:
exc_sid=["Vidarabine", "Insulin beef", "Trospium", "Gliclazide", "Etoricoxib", "Interferon alfa-2b", "Eplerenone", "Alfuzosin", "Insulin detemir", 
        "Maraviroc", "Eptifibatide", "Tolcapone", "Exemestane", "Insulin human", "Rivastigmine", "Nesiritide", "Valdecoxib", "Insulin pork", 
        "Tizanidine", "Ropinirole", "Fenofibrate", "Fluvoxamine", "Citalopram", "Timolol", "Amlodipine", "Duloxetine", "Letrozole", "Bicalutamide", 
        "Escitalopram", "Propafenone", "Anastrozole", "Pregabalin", "Venlafaxine", "Rabeprazole", "Fosinopril", "Celecoxib", "Nilotinib", "Naproxen",
"Metoprolol", "Varenicline", "Memantine", "Raloxifene", "Testosterone", "Dipyridamole", "Cisplatin", "Adenosine", "Vincristine", "Carbamazepine"]+[
"Phenyltoloxamine", "Potassium citrate", "Potassium Iodide", "Triethylenetetramine", "Nandrolone", "Articaine", "Methscopolamine", "Perazine", 
 "Etidocaine", "Trolnitrate", "Dexchlorpheniramine", "Fosamprenavir", "Dihydrocodeine", "Cevimeline", "Chlorotrianisene", "Mestranol", "Quinestrol",
 "Noscapine", "Bupivacaine", "Prilocaine", "Leuprolide", "Nabumetone", "Ketorolac", "Fenoprofen", "Mefenamic acid", "Oxaprozin", "Nylidrin", 
 "Isopropamide", "Milrinone", "Dexbrompheniramine", "Triprolidine", "Carbinoxamine", "Dimenhydrinate", "Alimemazine", "Clofedanol", "Chlorcyclizine", 
 "Chlorthalidone", "Guaiacol", "Isosorbide dinitrate", "Etonogestrel", "Desogestrel", "Ethynodiol diacetate", "Norelgestromin", "Dienogest", 
 "Brompheniramine", "Papaverine", "Minoxidil", "Methylphenidate", "Modafinil", "Phenylephrine", "Phentolamine", "Sibutramine", "Dexmethylphenidate", 
 "Promazine", "Perphenazine", "Acetophenazine", "Methotrimeprazine", "Oxycodone", "Butorphanol", "Hydrocodone", "Alvimopan", "Levonorgestrel", 
 "Fluoxymesterone", "Drospirenone", "Almotriptan", "Naratriptan", "Rizatriptan", "Calcium chloride", "Maraviroc", "Iloperidone", "Cyclizine", 
 "Entacapone", "Carbidopa", "Febuxostat", "Dyphylline", "Hydralazine", "Isoprenaline", "Tromethamine", "Misoprostol", "Tadalafil", 
 "Isosorbide mononitrate", "Mexiletine", "Pheniramine", "Asenapine", "Meprobamate", "Chlordiazepoxide", "Aripiprazole", "Dobutamine", "Synephrine", 
 "Phenylpropanolamine", "Prochlorperazine", "Maprotiline", "Doxylamine", "Propiomazine", "Dextromethorphan", "Atomoxetine", "Guaifenesin", "Ketamine", 
 "Capsaicin", "Ferrous fumarate", "Pentoxyverine", "Amrinone", "Ephedrine", "Potassium chloride", "Dextroamphetamine", "Glimepiride", "Isradipine", 
 "Glipizide", "Mirtazapine", "Pseudoephedrine", "Mepyramine", "Tretinoin", "Epirubicin", "Rosiglitazone", "Repaglinide", "Lisdexamfetamine",
 "Eletriptan", "Ziprasidone", "Lidocaine", "Zolmitriptan", "Amitriptyline", "Olanzapine", "Fluoxetine", "Chlorpromazine", "Triflupromazine", 
 "Nortriptyline", "Hydroxyzine", "Medroxyprogesterone acetate", "Fluphenazine", "Amprenavir", "Norethisterone", "Risperidone", "Scopolamine", 
 "Vardenafil", "Felodipine", "Propafenone", "Estramustine", "Quetiapine", "Paliperidone", "Stanolone", "Butabarbital", "Riboflavin", "Tolmetin", 
 "Sulindac", "Etodolac", "Diflunisal", "Caffeine", "Diethylstilbestrol", "Erlotinib", "Tegaserod", "Dasatinib", "Norgestimate", "Nifedipine", 
 "Tranexamic acid", "Celecoxib", "Flurbiprofen", "Meloxicam", "Ozanimod", "Pazopanib", "Sumatriptan", "Ethinylestradiol", "Piroxicam", "Estrone", 
 "Naproxen", "Ibuprofen", "Sitagliptin", "Methyltestosterone", "Theophylline", "Ketoprofen", "Glyburide", "Bupropion", "Codeine", "Epinephrine", 
 "Chlorpheniramine", "Amphetamine", "Nicotine", "Lansoprazole", "Clonidine", "Metamfetamine", "Testosterone cypionate", "Testosterone enanthate", 
 "Sildenafil", "Saxagliptin", "Levodopa", "Furosemide", "Phenobarbital", "Guanfacine", "Flecainide", "Indomethacin", "Lovastatin", "Progesterone", 
 "Testosterone", "Simvastatin", "Dipyridamole", "Pioglitazone", "Nefazodone", "Paclitaxel", "Phenytoin", "Liothyronine", "Levothyroxine", "Diclofenac", 
 "Estradiol", "Meclofenamic acid", "Estradiol cypionate", "Estradiol valerate", "Digoxin", "Ribavirin", "Inositol nicotinate", "Levomefolic acid", 
 "Acyclovir", "Sorafenib", "Metformin", "Sodium citrate", "Inositol"]+["Protamine", "Ioxaglic acid", "Calcitonin porcine", "Parnaparin", "Reviparin", 
"Telavancin", "Iodixanol", "Sorbitol", "Ioversol", "Ioxilan", 
 "Iopromide", "Cholestyramine", "Colestipol", "Iopamidol", "Mannitol", "Nefopam", "Insulin beef", "Enfuvirtide", "Bivalirudin", "Argatroban", 
 "Cevimeline", "Trospium", "Nilutamide", "Vinorelbine", "Esmolol", "Leuprolide", "Goserelin", "Gliclazide", "Rescinnamine", "Tiagabine", "Nabumetone", 
 "Tiaprofenic acid", "Etoricoxib", "Granisetron", "Capecitabine", "Methylergometrine", "Indapamide", "Salmon calcitonin", "Milrinone", "Anagrelide",
 "Nicorandil", "Sitaxentan", "Nitroglycerin", "Isosorbide dinitrate", "Hyoscyamine", "Pilocarpine", "Travoprost", "Minoxidil", "Methylphenidate", 
 "Interferon alfa-2b", "Eplerenone", "Lercanidipine", "Betaxolol", "Sotalol", "Metipranolol", "Bexarotene", "Porfimer sodium", "Alfuzosin", 
 "Phentolamine", "Fenoldopam", "Bretylium", "Moxifloxacin", "Trovafloxacin", "Sibutramine", "Dexmethylphenidate", "Mycophenolic acid", "Naltrexone", 
 "Bemiparin", "Risedronic acid", "Zaleplon", "Ondansetron", "Almotriptan", "Naratriptan", "Rizatriptan", "Insulin detemir", "Maraviroc", "Regadenoson", 
 "Streptokinase", "Warfarin", "Ifosfamide", "Salbutamol", "Pentosan polysulfate", "Irbesartan", "Pentoxifylline", "Tinzaparin", "Dalteparin", 
 "Tolcapone", "Entacapone", "Exemestane", "Insulin human", "Rivastigmine", "Rasagiline", "Moclobemide", "Febuxostat", "Pentostatin", "Hydralazine", 
 "Isoprenaline", "Glatiramer", "Muromonab", "Phentermine", "Teriparatide", "Tadalafil", "Gadopentetic acid", "Isosorbide mononitrate", "Zolpidem", 
 "Valdecoxib", "Dorzolamide", "Brinzolamide", "Mexiletine", "Insulin pork", "Mesalazine", "Nadroparin", "Aripiprazole", "Dobutamine", "Tizanidine", 
 "Pergolide", "Thalidomide", "Ropinirole", "Procarbazine", "Tramadol", "Milnacipran", "Chenodeoxycholic acid", "Human calcitonin", "Apomorphine", 
 "Ephedrine", "Gabapentin", "Acamprosate", "Dextroamphetamine", "Glimepiride", "Isradipine", "Bosentan", "Atropine", "Fenofibrate", "Mirtazapine", 
 "Pseudoephedrine", "Tiotropium", "Epirubicin", "Repaglinide", "Fluvoxamine", "Citalopram", "Eletriptan", "Ziprasidone", "Zolmitriptan", 
 "Dihydroergotamine", "Clozapine", "Timolol", "Fluoxetine", "Lenalidomide", "Toremifene", "Nicardipine", "Paroxetine", "Risperidone", "Fentanyl", 
 "Vardenafil", "Letrozole", "Felodipine", "Selegiline", "Bicalutamide", "Carvedilol", "Terazosin", "Escitalopram", "Propafenone", "Anastrozole", 
 "Quetiapine", "Bepridil", "Darunavir", "Ixabepilone", "Indacaterol", "Dabigatran etexilate", "Fesoterodine", "Candesartan", "Pregabalin", "Fluorescein"
 , "Captopril", "Venlafaxine", "Omeprazole", "Riluzole", "Donepezil", "Buprenorphine", "Leflunomide", "Rabeprazole", "Dasatinib", "Cerivastatin", 
 "Tamoxifen", "Eprosartan", "Telmisartan", "Lomefloxacin", "Nifedipine", "Ofloxacin", "Sparfloxacin", "Ramipril", "Tranexamic acid", "Fosinopril", 
 "Trandolapril", "Benazepril", "Lisinopril", "Perindopril", "Cilazapril", "Celecoxib", "Rofecoxib", "Flurbiprofen", "Meloxicam", "Labetalol", 
 "Nilotinib", "Vinblastine", "Etravirine", "Sumatriptan", "Ursodeoxycholic acid", "Oseltamivir", "Pantoprazole", "Piroxicam", "Pemetrexed", 
 "Esomeprazole", "Naproxen", "Quinapril", "Hydrochlorothiazide", "Melatonin", "Fluorouracil", "Enalapril", "Dofetilide", "Metoprolol", "Allopurinol", 
 "Amiloride", "Amiodarone", "Varenicline", "Reserpine", "Indinavir", "Quinine", "Lamotrigine", "Doxazosin", "Clopidogrel", "Ranitidine", "Quinidine", 
 "Amphetamine", "Pramipexole", "Lansoprazole", "Prazosin", "Irinotecan", "Formoterol", "Sildenafil", "Daunorubicin", "Glycine", "Latanoprost", 
 "Memantine", "Flecainide", "Pravastatin", "Bortezomib", "Progesterone", "Testosterone", "Dipyridamole", "Pioglitazone", "Nefazodone", "Docetaxel", 
 "Everolimus", "Ambrisentan", "Liothyronine", "Levothyroxine", "Dextrothyroxine", "Diclofenac", "Dopamine", "Carboplatin", "Tacrolimus", "Imatinib", 
 "Fludarabine", "Niacin", "Ciprofloxacin", "Verapamil", "Levofloxacin", "Doxorubicin", "Metformin", "Losartan", "Topiramate"]+["Almotriptan", 
"Naratriptan", "Rizatriptan", "Frovatriptan", "Eletriptan", "Zolmitriptan", "Dihydroergotamine", "Diltiazem", "Amlodipine", "Nicardipine", "Ergotamine",
"Felodipine", "Carvedilol", "Nifedipine", "Sumatriptan", "Naproxen", "Nicotine", "Atorvastatin", "Verapamil"]+["Protamine", "Ioxaglic acid", 
"Calcitonin porcine", "Parnaparin", "Reviparin", "Telavancin", "Iodixanol", "Sorbitol", "Ioversol", "Ioxilan", "Iopromide", "Cholestyramine", 
"Colestipol", "Iopamidol", "Mannitol", "Nefopam", "Insulin beef", "Enfuvirtide", "Bivalirudin", "Argatroban", "Cevimeline", "Trospium", "Nilutamide",
"Vinorelbine", "Esmolol", "Leuprolide", "Goserelin", "Gliclazide", "Rescinnamine", "Tiagabine", "Nabumetone", "Tiaprofenic acid", "Etoricoxib", 
"Granisetron", "Capecitabine", "Methylergometrine", "Indapamide", "Salmon calcitonin", "Milrinone", "Anagrelide", "Nicorandil", "Sitaxentan", 
"Nitroglycerin", "Isosorbide dinitrate", "Hyoscyamine", "Pilocarpine", "Travoprost", "Minoxidil", "Methylphenidate", "Interferon alfa-2b", "Eplerenone",
"Lercanidipine", "Betaxolol", "Sotalol", "Metipranolol", "Bexarotene", "Porfimer sodium", "Alfuzosin", "Phentolamine", "Fenoldopam", "Bretylium", 
"Moxifloxacin", "Trovafloxacin", "Sibutramine", "Dexmethylphenidate", "Mycophenolic acid", "Naltrexone", "Bemiparin", "Risedronic acid", "Zaleplon",
"Ondansetron", "Almotriptan", "Naratriptan", "Rizatriptan", "Frovatriptan", "Insulin detemir", "Maraviroc", "Regadenoson", "Streptokinase", "Warfarin", 
"Ifosfamide", "Salbutamol", "Pentosan polysulfate", "Irbesartan", "Pentoxifylline", "Tinzaparin", "Dalteparin", "Tolcapone", "Entacapone", "Exemestane",
"Insulin human", "Rivastigmine", "Rasagiline", "Moclobemide", "Febuxostat", "Pentostatin", "Hydralazine", "Isoprenaline", "Glatiramer", "Muromonab", 
"Phentermine", "Teriparatide", "Tadalafil", "Gadopentetic acid", "Isosorbide mononitrate", "Zolpidem", "Valdecoxib", "Dorzolamide", "Brinzolamide", 
"Mexiletine", "Insulin pork", "Mesalazine", "Nadroparin", "Aripiprazole", "Dobutamine", "Tizanidine", "Pergolide", "Thalidomide", "Ropinirole", 
"Procarbazine", "Tramadol", "Milnacipran", "Chenodeoxycholic acid", "Human calcitonin", "Apomorphine", "Ephedrine", "Gabapentin", "Acamprosate", 
"Dextroamphetamine", "Glimepiride", "Isradipine", "Bosentan", "Atropine", "Fenofibrate", "Mirtazapine", "Pseudoephedrine", "Tiotropium", "Epirubicin", 
"Repaglinide", "Fluvoxamine", "Citalopram", "Eletriptan", "Ziprasidone", "Zolmitriptan", "Dihydroergotamine", "Diltiazem", "Clozapine", "Timolol", 
"Amlodipine", "Fluoxetine", "Lenalidomide", "Toremifene", "Nicardipine", "Ergotamine", "Paroxetine", "Risperidone", "Fentanyl", "Vardenafil",
"Letrozole", "Felodipine", "Selegiline", "Bicalutamide", "Carvedilol", "Terazosin", "Escitalopram", "Propafenone", "Anastrozole", "Quetiapine",
"Bepridil", "Darunavir", "Ixabepilone", "Indacaterol", "Dabigatran etexilate", "Fesoterodine", "Candesartan", "Pregabalin", "Fluorescein", "Captopril", 
"Venlafaxine", "Omeprazole", "Riluzole", "Donepezil", "Buprenorphine", "Leflunomide", "Rabeprazole", "Dasatinib", "Cerivastatin", "Tamoxifen", 
"Eprosartan", "Telmisartan", "Lomefloxacin", "Nifedipine", "Ofloxacin", "Sparfloxacin", "Ramipril", "Tranexamic acid", "Fosinopril", "Trandolapril", 
"Benazepril", "Lisinopril", "Perindopril", "Cilazapril", "Celecoxib", "Rofecoxib", "Flurbiprofen", "Meloxicam", "Labetalol", "Nilotinib", "Vinblastine",
"Etravirine", "Sumatriptan", "Ursodeoxycholic acid", "Oseltamivir", "Pantoprazole", "Piroxicam", "Pemetrexed", "Esomeprazole", "Naproxen", "Quinapril", 
"Hydrochlorothiazide", "Melatonin", "Fluorouracil", "Enalapril", "Dofetilide", "Metoprolol", "Allopurinol", "Amiloride", "Amiodarone", "Varenicline", 
"Reserpine", "Indinavir", "Quinine", "Lamotrigine", "Doxazosin", "Clopidogrel", "Ranitidine", "Quinidine", "Amphetamine", "Nicotine", "Pramipexole", 
"Lansoprazole", "Prazosin", "Irinotecan", "Formoterol", "Sildenafil", "Daunorubicin", "Glycine", "Latanoprost", "Memantine", "Flecainide", "Pravastatin"
, "Bortezomib", "Progesterone", "Testosterone", "Dipyridamole", "Atorvastatin", "Pioglitazone", "Nefazodone", "Docetaxel", "Everolimus", "Ambrisentan", 
"Liothyronine", "Levothyroxine", "Dextrothyroxine", "Diclofenac", "Dopamine", "Carboplatin", "Tacrolimus", "Imatinib", "Fludarabine", "Niacin", 
                                        "Ciprofloxacin", "Verapamil", "Levofloxacin", "Doxorubicin", "Metformin", "Losartan", "Topiramate"]




In [242]:
list_all_med = list(set(Med.y_name))

In [ ]:
list_of_pos_neg = HF_cont+exc_sid+HF_ind

In [245]:
list_uknown_med = [item for item in list_all_med if item not in list_of_pos_neg]

In [246]:
med_embeddigns_pos = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('myocardial infarction'))) for key in HF_ind if key in medication_embeddings]
med_embeddigns_neg = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('myocardial infarction'))) for key in HF_cont if key in medication_embeddings]
uknown_medication_embeddings = [np.concatenate((medication_embeddings[key],diseases_embeddings.get('myocardial infarction'))) for key in list_uknown_med if key in medication_embeddings]
HF_embeddings = diseases_embeddings.get('myocardial infarction')

In [247]:
med_disease_pos = pd.DataFrame(np.array(med_embeddigns_pos))
med_disease_pos['Label'] = 1
med_disease_neg = pd.DataFrame(np.array(med_embeddigns_neg))
med_disease_neg['Label'] = 0

In [250]:
med_disease_pos_neg = pd.concat([med_disease_pos,med_disease_neg])

In [251]:
med_disease_pos_neg.reset_index(drop=True, inplace=True)

In [252]:
uknown_med_names= [key for key in list_uknown_med if key in medication_embeddings]
uknown_med_embeddigns_df = pd.DataFrame(np.array(uknown_medication_embeddings))

In [253]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
import random 

random.seed(123456)
np.random.seed(123456)
X = med_disease_pos_neg.iloc[:,0:-1]
y = med_disease_pos_neg.iloc[:,-1]
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=20)

In [254]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 5, 10, 50, 100, 200, 300, 500],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

# Initialize the classifier
random.seed(123)
svm = SVC(probability=True, random_state=42)

# Initialize GridSearchCV
random.seed(123)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_svm = grid_search.best_estimator_

print("Best parameters found: ", best_params)

# Evaluate the best estimator
y_pred_svm = best_svm.predict(X_test)
y_pred_proba_svm = best_svm.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba_svm))


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters found:  {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.7272727272727273
ROC AUC: 0.7606837606837606


In [255]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

random.seed(123)
# Define the tuned parameters (example parameters, replace with your actual tuned values)
tuned_params = {
    'C': 100,        # Regularization parameter
    'gamma': 1, # Kernel coefficient
    'kernel': 'rbf' # Kernel type
}

# Initialize the classifier with tuned parameters
svm = SVC(probability=True, random_state=42, **tuned_params)

# Train the classifier
svm.fit(X_train, y_train)

# Predict the test set labels
y_pred_svm = svm.predict(X_test)
y_pred_proba_svm = svm.predict_proba(X_test)[:, 1]

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test,y_pred_proba_svm))


Accuracy: 0.7272727272727273
ROC AUC: 0.7606837606837606


In [258]:
#svm.decision_function(uknown_med_embeddigns_df.iloc[i:i+1,:])

In [259]:
svm_my_dict = {}
for i in range(0,uknown_med_embeddigns_df.shape[0]):
    y_pred_svm_p = svm.predict(uknown_med_embeddigns_df.iloc[i:i+1,:])
    y_pred_proba_svm_p = svm.predict_proba(uknown_med_embeddigns_df.iloc[i:i+1,:])[:, 1]
    print(uknown_med_names[i],y_pred_svm_p[0],y_pred_proba_svm_p[0])
    if y_pred_svm_p[0]>0:
        svm_my_dict[uknown_med_names[i]] =y_pred_proba_svm_p[0]

pos_prd_prob_svm = dict(sorted(svm_my_dict.items(), key=lambda item: item[1],reverse=True))

Denileukin diftitox 1 0.9999810225156445
Etanercept 1 0.7862947040816722
Lisuride 0 0.20875529774999818
Sunitinib 0 0.473333356332977
Trimethaphan 1 0.9271059470769719
Isavuconazonium 1 0.9807366254099361
Cefmetazole 1 0.999986879567989
Ethosuximide 0 0.11309682752457807
Silicon dioxide 1 0.5252092211134847
Aminoglutethimide 0 0.19766646438288993
Hydromorphone 1 0.8070741646537442
Nepafenac 1 0.9315122984053993
Cabozantinib 1 0.9902011507247438
Busulfan 1 0.6667331377165395
Gliquidone 1 0.86079194622953
Trimethadione 0 0.20062568248258994
Citrulline 0 0.059080171876341105
Canakinumab 1 0.9803868958430794
Ceftriaxone 0 0.043400203489179084
Limaprost 1 0.7397506990177307
Dasabuvir 0 0.2803854142988766
Nafarelin 1 0.8880430299068065
Leucine 1 0.73312953162951
Polyethylene glycol 3500 1 0.9224879743889633
Phylloquinone 0 0.40651141588169454
Cenegermin 1 0.9856581051201309
Cladribine 0 0.1627341572823227
Naftopidil 1 0.8729929129134948
Vilazodone 0 0.30310531964278303
Tetryzoline 1 0.934134